In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

from Load_Predictions import *
from downstream_models import *
from Featurize import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
testdf = pd.read_csv('Benchmark-Labeled-Data/data_train.csv')
test_metadata = pd.read_csv('../RawCSV/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')

y_true = test_merged.y_act.values.tolist()

7936 4233


In [3]:
test_merged

,Record_id,Attribute_name,y_act,total_vals,num_nans,%_nans,num_of_dist_val,%_dist_val,mean,std_dev,...,mean_char_count,stdev_char_count,mean_whitespace_count,stdev_whitespace_count,mean_delim_count,stdev_delim_count,is_list,is_long_sentence,name,link
0,33,Area,categorical,21477,0,0.000000,174,0.810169,0.000000,0.000000,...,10.0,4.816638,0.4,0.800000,0.4,0.800000,False,False,FAO.csv,https://www.kaggle.com/dorbicycle/world-foodfe...
1,33,Area Code,categorical,21477,0,0.000000,174,0.810169,125.449411,72.866452,...,1.0,0.000000,0.0,0.000000,0.0,0.000000,False,False,FAO.csv,https://www.kaggle.com/dorbicycle/world-foodfe...
2,33,Element,categorical,21477,0,0.000000,2,0.009312,0.000000,0.000000,...,4.0,0.000000,0.0,0.000000,0.0,0.000000,False,False,FAO.csv,https://www.kaggle.com/dorbicycle/world-foodfe...
3,33,Element Code,categorical,21477,0,0.000000,2,0.009312,5211.687154,146.816661,...,4.0,0.000000,0.0,0.000000,0.0,0.000000,False,False,FAO.csv,https://www.kaggle.com/dorbicycle/world-foodfe...
4,33,Item,categorical,21477,0,0.000000,115,0.535457,0.000000,0.000000,...,19.6,2.244994,2.0,0.000000,2.0,0.000000,False,False,FAO.csv,https://www.kaggle.com/dorbicycle/world-foodfe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7931,4180,%,embedded-number,94,39,41.489362,41,43.617021,0.000000,0.000000,...,4.6,0.800000,0.0,0.000000,0.0,0.000000,False,False,Veal & Calf History.csv,https://www.kaggle.com/usda/the-national-summa...
7932,4181,date,datetime,366,0,0.000000,366,100.000000,0.000000,0.000000,...,6.0,0.000000,0.0,0.000000,0.0,0.000000,False,False,weather_data_nyc_centralpark_2016(1).csv,https://www.kaggle.com/mathijs/weather-data-in...
7933,4185,notes,sentence,2023,5,0.247158,311,15.373208,0.000000,0.000000,...,32.8,21.113029,4.8,3.600000,4.8,3.600000,False,False,klavern.csv,https://www.kaggle.com/jboysen/mapping-the-kkk
7934,4201,Purposes,sentence,21,1,4.761905,20,95.238095,0.000000,0.000000,...,52.2,29.294368,7.4,4.223742,7.4,4.223742,False,False,Schemes of india.csv,https://www.kaggle.com/robinreni/inda-basic-da...


In [4]:
dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]

In [5]:
table_names = ['Record_id', 'Attribute_name', 'y_act', 'total_vals', 'num_nans',
           '%_nans', 'num_of_dist_val', '%_dist_val', 'mean', 'std_dev', 'min_val',
           'max_val', 'has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count',
           'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total',
           'mean_char_count', 'stdev_char_count', 'mean_whitespace_count',
           'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count',
           'is_list', 'is_long_sentence', 
           'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']

In [6]:
sample_size_list = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1]
rand_seed = 99

results = []

for sample_percent in sample_size_list:
    
    df_data = pd.DataFrame()
    
    prv_csv_name, csv_name = '', ''
    
    exception_indices = []
    error_count = 0

    for index, row in test_merged.iterrows():  
        
        if index%50==0:
            print(index)
            
        col = str(row['Attribute_name'])
        prv_csv_name = csv_name
        csv_name = '../RawCSV/RawCSVFiles/' + row['name']

        if prv_csv_name != csv_name:
            df = pd.read_csv(csv_name,encoding='latin1')

        try:
            total_vals=df[[col]].shape[0]
            sample_size = int(total_vals*sample_percent)

            df_sample = df[[col]].sample(n=sample_size, random_state=rand_seed)

            dataFeaturized = FeaturizeFile(df_sample)
            dataFeaturized1 = ProcessStats(dataFeaturized)
            dataFeaturized2 = FeatureExtraction(dataFeaturized,dataFeaturized1,0)
            dataFeaturized2 = dataFeaturized2.fillna(0)

            dataFeaturized2.insert(0, 'y_act', row['y_act'])
            dataFeaturized2.insert(0, 'Attribute_name', row['Attribute_name'])
            dataFeaturized2.insert(0, 'Record_id', row['Record_id'])
            labeled_row = dataFeaturized2.iloc[:, :28]

            sample_value_df = df[[col]].sample(5, random_state=rand_seed)
            for sample_value in range(len(sample_value_df[col].values)):
                col_name = f"sample_{sample_value+1}"
                labeled_row.insert(12, col_name, sample_value_df[col].values[sample_value])

            df_data = df_data.append(labeled_row)
            
        except Exception as e: 
#             original_row = pd.DataFrame()
#             for col_name in table_names:
#                 if (col_name in row.index):
#                     original_row[col_name] = [row[col_name]]

#             df_data = df_data.append(original_row)
            print('Error')
            error_count += 1
            
            
    df_data = df_data[table_names]
    print(df_data.columns)
    print(error_count)
    df_data.to_csv(f"DescriptiveStatisticSubset-Data/{sample_percent}_sample_data_train.csv", index=False)


0
50
100


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


150
200
250
300
350
400
450
500
550
600
650
700
750
800
Error
Error
Error
Error
850
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
900
950


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1000


...

6550
Error
6600
Error
6650
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
6700
6750
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
6800
6850
Error
Error
Error
Error
Error
Error
Error
Error
Error
6900
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
6950
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7000
Error
Error
Error
Error
7050
Error
7100
Error
Error
7150
Error
Error
Error
7200
Error
Error
Error
Error
Error
7250
Error
Error
Error
Error
Error
Error
Error
7300
7350
Error
Error
7400
Error
Error
7450


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1212: UnknownTimezoneWarning: tzname AST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7500
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7550
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7600
Error
Error
Error
Error
Error
Error


C:\Users\Andrew\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1212: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


7650
Error
Error
7700
7750
7800
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7850
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
7900
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Index(['Record_id', 'Attribute_name', 'y_act', 'total_vals', 'num_nans',
       '%_nans', 'num_of_dist_val', '%_dist_val', 'mean', 'std_dev', 'min_val',
       'max_val', 'has_delimiters', 'has_url', 'has_email', 'has_date',
       'mean_word_count', 'std_dev_word_count', 'mean_stopword_total',
       'stdev_

In [7]:
df_data

,Record_id,Attribute_name,y_act,total_vals,num_nans,%_nans,num_of_dist_val,%_dist_val,mean,std_dev,...,stdev_whitespace_count,mean_delim_count,stdev_delim_count,is_list,is_long_sentence,sample_1,sample_2,sample_3,sample_4,sample_5
0,33,Area,categorical,2147,0,0.000000,174,8.104332,0.000000,0.000000,...,1.264911,1.0,1.264911,False,False,Finland,Luxembourg,Trinidad and Tobago,United Republic of Tanzania,Canada
0,33,Area Code,categorical,2147,0,0.000000,174,8.104332,123.564974,73.147125,...,0.000000,0.0,0.000000,False,False,67,256,220,215,33
0,33,Element,categorical,2147,0,0.000000,2,0.093153,0.000000,0.000000,...,0.000000,0.0,0.000000,False,False,Food,Food,Food,Food,Food
0,33,Element Code,categorical,2147,0,0.000000,2,0.093153,5217.905915,151.679379,...,0.000000,0.0,0.000000,False,False,5142,5142,5142,5142,5142
0,33,Item,categorical,2147,0,0.000000,115,5.356311,0.000000,0.000000,...,1.019804,1.4,1.019804,True,False,"Marine Fish, Other","Cereals, Other","Beverages, Fermented",Milk - Excluding Butter,Cream
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4180,%,embedded-number,9,5,55.555556,4,44.444444,0.000000,0.000000,...,0.000000,0.0,0.000000,False,False,7.7%,NaN,10.7%,51.8%,NaN
0,4181,date,datetime,36,0,0.000000,36,100.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,False,False,8-1-2016,5-8-2016,21-2-2016,14-7-2016,11-4-2016
0,4185,notes,sentence,202,1,0.495050,26,12.871287,0.000000,0.000000,...,8.777243,10.6,8.777243,False,True,No Notes,No Notes,No Notes,No Notes,No Notes
0,4201,Purposes,sentence,2,0,0.000000,2,100.000000,0.000000,0.000000,...,2.939388,11.4,2.939388,False,True,provides educational opportunities to enhance ...,the creation of large and quality oriented tr...,to provide skills training and enhance the emp...,to empower and instill confidence among minori...,providing financial assistance for skill train...
